In [28]:
from src.utils import read_data, missing_values_table
from src.regression_algorithms import run_reg_pred, algo_map
from datetime import datetime, timedelta
import calendar
import pandas as pd

In [21]:
from IPython.display import display_html, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def display_dfs(dfs, gap=50, justify='center'):
    html = ""
    for title, df in dfs.items():  
        df_html = df._repr_html_()
        cur_html = f'<div> <h3>{title}</h3> {df_html}</div>'
        html +=  cur_html
    html= f"""
    <div style="display:flex; gap:{gap}px; justify-content:{justify};">
        {html}
    </div>
    """
    display_html(html, raw=True)

In [22]:
dat = read_data()

In [26]:
percentiles = [.01, .025, .05, .1, .25, .5, .75, .95, .975, .99, .999]
dfs1 = {'Head': dat.head(5), 'Tail': dat.tail(5)}
dfs2 = {'DTypes': pd.DataFrame(dat.dtypes), 'Describe': dat.describe(percentiles=percentiles).T}
display_dfs(dfs1, justify='flex-start')
display_dfs(dfs2, justify='flex-start')
display(missing_values_table(dat))

,Date,Consumption
0,2015-04-01,357.073
1,2015-04-02,341.931
2,2015-04-03,318.777
3,2015-04-04,312.494
4,2015-04-05,270.837
,Date,Consumption
1822,2020-03-27,237.171
1823,2020-03-28,229.744
1824,2020-03-29,297.636
1825,2020-03-30,323.811


DTypes 

 
 
 
 
 0 
 
 
 
 
 Date 
 datetime64[ns] 
 
 
 Consumption 
 float64 
 
 
 
 Describe 

 
 
 
 
 count 
 mean 
 std 
 min 
 1% 
 2.5% 
 5% 
 10% 
 25% 
 50% 
 75% 
 95% 
 97.5% 
 99% 
 99.9% 
 max 
 
 
 
 
 Consumption 
 1827.0 
 257.885096 
 100.993368 
 82.675 
 109.32474 
 118.3223 
 129.2541 
 141.7106 
 174.6725 
 242.719 
 327.841 
 442.6629 
 475.45865 
 523.17992 
 574.534678 
 589.269

Your selected dataframe has 2 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [33]:
preds = {"algo": [], "preds": [], "accuracy": [], "mse": []}
for algo_name, algo in algo_map.items():
    if 'vote_' not in algo_name:  # ignoring voting algos
        res = run_reg_pred(dat[["Consumption"]], "Consumption", 365, algo_name=algo_name)
        preds["preds"].append(res["preds"])
        preds["accuracy"].append(res["accuracy"])
        preds["mse"].append(res["mse"])
        preds["algo"].append(algo_name)

preds_df = pd.DataFrame(preds)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\datas\PycharmProjects\main\M2\energy-demand-forecast\src\regression_algorithms.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[pred_target] = data[target].shift(-pred_range)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

In [34]:
preds_df.sort_values(by=["accuracy"], ascending=False)

,algo,preds,accuracy,mse
10,svr_poly_scale_coef,"[244.56396594069656, 273.7797694933221, 279.06...",0.674095,3066.169811
9,svr_poly_auto_coef,"[243.34612282652415, 272.74906548748163, 278.1...",0.648569,4022.553129
17,randomforest_0,"[264.05040105613114, 292.1108336811592, 292.66...",0.641169,3713.025996
8,svr_poly_coef,"[243.30866048600063, 272.9896287601334, 278.37...",0.639167,3716.129502
20,grad_boost_1,"[257.61241701947006, 293.9983183475211, 295.00...",0.635409,3748.553922
19,randomforest_1,"[247.54743923962297, 301.43097919990896, 302.8...",0.629236,3762.241250
18,grad_boost_0,"[252.9637717213645, 296.139175039923, 302.3810...",0.617250,4086.066057
16,ridge,"[241.4315693758183, 266.631496596954, 271.3419...",0.615355,4347.555503
11,svr_poly_lerr_coef,"[242.88749107780203, 263.5246622154286, 267.18...",0.600818,4071.915029
1,svr_linear,"[244.11654821626445, 276.9715262402144, 282.45...",0.591201,3897.912603
